# Module 1 - Homework

## Q1. Running Elastic

In [3]:
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch('http://localhost:9200')

In [7]:
es_client.info()

ObjectApiResponse({'name': '8edfe3cca075', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'J5fG_xtaTiec5vx24XJ6cg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
## Getting the data

In [11]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [12]:
## Q2. Indexing the data

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework_course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [16]:
from tqdm.auto import tqdm

In [17]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Question: Which function do you use for adding your data to elastic?

insert
index
put
add

#### Answer

index

## Q3. Searching


In [28]:
query = 'How do execute a command on a Kubernetes pod?'

In [29]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response


ObjectApiResponse({'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'homework_course-questions', '_id': '2b5zdZcBD26SI8ZwenXO', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'homework_course-questions', '_id': 'aL5zdZcBD26SI8ZwfHZ8', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'q

### Question: What's the score for the top ranking result?

84.50
64.50
44.50
24.50

### Answer: 44.50556

In [25]:
## Q4; Filtering

In [76]:
query = 'How do copy a file to a Docker container?'

In [81]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs


### Question: Return 3 results. What's the 3rd question returned by the search engine?

### Answer: 'question': 'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [82]:
response = elastic_search(query)

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
for doc in response:
        context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"
    
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:{context}
""".strip()

prompt = prompt_template.format(question=query, context=context).strip()

In [83]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_i

In [84]:
len(prompt)

1445

In [61]:
## Question: What's the length of the resulting prompt? (use the len function)

In [85]:
## Answer: 1446 (closest one)

In [63]:
## Q6. Tokens

In [86]:
pip install tiktoken


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [70]:
import tiktoken

In [87]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
print("The number of tokens in the resulting prompt is:", num_tokens)

The number of tokens in the resulting prompt is: 320
